In [154]:
import pandas as pd
import plotly.express as px
import re
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.options.display.max_colwidth = 2000

In [155]:
# Functions

def column_name_cleaning(table):
    table.columns = table.columns.str.lower()
    table.columns = [re.sub('(,)|(:)|(-)', '', x) for x in table.columns]
    table.columns = [re.sub('(_)', ' ', x) for x in table.columns]
    table.columns = [x.strip(' ') for x in table.columns]
    
    # patient id column name normalization
    if 'subject number' in table.columns: 
        table = table.rename(mapper={'subject number': 'patient id'}, axis=1)
    elif 'patient id' in table.columns: 
        table = table.rename(mapper={'patient id': 'patient id'}, axis=1)

def str_column_cleaning(table):
    for col in table.columns:
        if table[col].dtype == object:
            table[col] = table[col].str.lower()
            table[col] = table[col].str.replace('(,)|(:)|(-)', '', regex=True)
            table[col] = table[col].str.replace('(_)', ' ', regex=True)
            table[col] = table[col].str.strip(' ')

# Loading

In [156]:
dir = '../data/cerebral_perfusion_data/data_description/'
filename_dat = 'GE-75_data_summary_table.csv'
filename_desc = 'GE-75_data_dictionary.csv'
df = pd.read_csv(dir+filename_dat)
df_desc = pd.read_csv(dir+filename_desc, usecols=range(5))

In [157]:
df.head() #encoding problem, pandas and linux = unknown-8bit, vscode = utf8. Saved it as utf8 on vscode

,patient ID,study,Group,template,Diabetes Duration,HTN or not,HTN2,age,sex,Gender,HEIGHT (M),MASS (KG),BMI,global GM vol,global GM vol covered in template,L superior frontal gyrus (#21) GM,R superior frontal gyrus (#22) GM,L middle frontal gyrus (#23) GM,R middle frontal gyrus (#24) GM,L inferior frontal gyrus (#25) GM,R inferior frontal gyrus (#26) GM,L precentral gyrus (#27) GM,R precentral gyrus (#28) GM,L middle orbitofrontal gyrus (#29) GM,R middle orbitofrontal gyrus (#30) GM,L lateral orbitofrontal gyrus (#31) GM,R lateral orbitofrontal gyrus (#32) GM,L gyrus rectus (#33) GM,R gyrus rectus (#34) GM,L postcentral gyrus (#41) GM,R postcentral gyrus (#42) GM,L superior parietal gyrus (#43) GM,R superior parietal gyrus (#44) GM,L supramarginal gyrus (#45) GM,R supramarginal gyurs (#46) GM,L angular gyrus (#47) GM,R angular gyrus (#48) GM,L precuneus (#49) GM,R precuneus (#50) GM,L superior occipital gyrus (#61) GM,R superior occipital gyrus (#62) GM,L middle occipital gyrus (#63) GM,R middle occipital gyrus (#64) GM,L inferior occipital gyrus (#65) GM,R inferior occipital gyrus (#66) GM,L cuneus (#67) GM,R cuneus (#68) GM,L superior temporal gyrus (#81) GM,R superior temporal gyrus (#82) GM,L middle temporal gyrus (#83) GM,R middle temporal gyrus (#84) GM,L inferior temporal gyrus (#85) GM,R inferior temporal gyrus (#86) GM,L parahippocampal gyrus (#87) GM,R parahippocampal gyrus (#88) GM,L lingual gyrus (#89) GM,R lingual gyrus (#90) GM,L fusiform gyrus (#91) GM,R fusiform gyrus (#92) GM,L insular cortex (#101) GM,R insular cortex (#102) GM,L cingulate gyrus (#121) GM,R cingulate gyrus (#122) GM,L caudate (#161) GM,R caudate (#162) GM,L putamen (#163) GM,R putamen (#164) GM,L hippocampus (#165) GM,R hippocampus (#166) GM,cerebellum (#181) GM,brain stem (#182) GM,global WM vol,global WM vol covered in template,L superior frontal gyrus (#21) WM,R superior frontal gyrus (#22) WM,L middle frontal gyrus (#23) WM,R middle frontal gyrus (#24) WM,L inferior frontal gyrus (#25) WM,R inferior frontal gyrus (#26) WM,L precentral gyrus (#27) WM,R precentral gyrus (#28) WM,L middle orbitofrontal gyrus (#29) WM,R middle orbitofrontal gyrus (#30) WM,L lateral orbitofrontal gyrus (#31) WM,R lateral orbitofrontal gyrus (#32) WM,L gyrus rectus (#33) WM,R gyrus rectus (#34) WM,L postcentral gyrus (#41) WM,R postcentral gyrus (#42) WM,L superior parietal gyrus (#43) WM,R superior parietal gyrus (#44) WM,L supramarginal gyrus (#45) WM,R supramarginal gyurs (#46) WM,L angular gyrus (#47) WM,R angular gyrus (#48) WM,L precuneus (#49) WM,R precuneus (#50) WM,L superior occipital gyrus (#61) WM,R superior occipital gyrus (#62) WM,L middle occipital gyrus (#63) WM,R middle occipital gyrus (#64) WM,L inferior occipital gyrus (#65) WM,R inferior occipital gyrus (#66) WM,L cuneus (#67) WM,R cuneus (#68) WM,L superior temporal gyrus (#81) WM,R superior temporal gyrus (#82) WM,L middle temporal gyrus (#83) WM,R middle temporal gyrus (#84) WM,L inferior temporal gyrus (#85) WM,R inferior temporal gyrus (#86) WM,L parahippocampal gyrus (#87) WM,R parahippocampal gyrus (#88) WM,L lingual gyrus (#89) WM,R lingual gyrus (#90) WM,L fusiform gyrus (#91) WM,R fusiform gyrus (#92) WM,L insular cortex (#101) WM,R insular cortex (#102) WM,L cingulate gyrus (#121) WM,R cingulate gyrus (#122) WM,L caudate (#161) WM,R caudate (#162) WM,L putamen (#163) WM,R putamen (#164) WM,L hippocampus (#165) WM,R hippocampus (#166) WM,cerebellum (#181) WM,brain stem (#182) WM,global CSF vol,global CSF vol covered in template,L superior frontal gyrus (#21) CSF,R superior frontal gyrus (#22) CSF,L middle frontal gyrus (#23) CSF,R middle frontal gyrus (#24) CSF,L inferior frontal gyrus (#25) CSF,R inferior frontal gyrus (#26) CSF,L precentral gyrus (#27) CSF,R precentral gyrus (#28) CSF,L middle orbitofrontal gyrus (#29) CSF,R middle orbitofrontal gyrus (#30) CSF,L lateral orbitofrontal gyrus (#31) CSF,R lateral orbitofrontal gyrus (#32) CSF,L gyrus rectus (#33) CSF,R gyrus rectus (

In [158]:
df_desc.head()

,SUBJECT NUMBER,S0328,S0342,Variable description,units
0,NaN,NaN,NaN,NaN,NaN
1,STUDY,GE-75,GE-75,study number,NaN
2,COMPLETED VISITS STATUS,COMPLETED,COMPLETED,completed study status,NaN
3,HTN PATIENT MEDICAL HISTORY,NO,NO,hypertension patient history,NaN
4,GROUP 2,CONTROL,DM,group control - diabetes,NaN


# Exploration

The exploration part aims 
- to assess the relevance of the dataset for our use case: explore links between diabetes and cardiovascular diseases in order to the development of a future application to follow  diabetic patients.
- to analyze the consistency with the documentation available for the dataset
- to detect problems in the data: missing value, wrong format, renaming, etc.

3 categories of features have been identified to be potentially interesting: 
- demographics: age, gender, bmi, etc
- lifestyle: physical activity, smoking habits, diet
- medical record: previous cancer, stroke, 

In [159]:
print(df.shape)
print(len(pd.unique(df["patient ID"])))

(88, 1111)
88


In [160]:
print(df_desc.shape)
print(len(pd.unique(df_desc["SUBJECT NUMBER"])))

(1149, 5)
1143


This dataset contains 88 unique patients and 1111 features. This is inconsistent with number of patients found in their paper: 71 TD2M + 76 ctrl 

The description file contains more features than the data file

Let's have an overview of the features:

In [161]:
list(df.columns)

['patient ID',
 'study',
 'Group',
 'template',
 'Diabetes Duration',
 'HTN or not',
 'HTN2',
 'age',
 'sex',
 'Gender',
 'HEIGHT (M)',
 'MASS (KG)',
 'BMI',
 'global GM vol',
 'global GM vol covered in template',
 'L superior frontal gyrus (#21) GM',
 'R superior frontal gyrus (#22) GM',
 'L middle frontal gyrus (#23) GM',
 'R middle frontal gyrus (#24) GM',
 'L inferior frontal gyrus (#25) GM',
 'R inferior frontal gyrus (#26) GM',
 'L precentral gyrus (#27) GM',
 'R precentral gyrus (#28) GM',
 'L middle orbitofrontal gyrus (#29) GM',
 'R middle orbitofrontal gyrus (#30) GM',
 'L lateral orbitofrontal gyrus (#31) GM',
 'R lateral orbitofrontal gyrus (#32) GM',
 'L gyrus rectus (#33) GM',
 'R gyrus rectus (#34) GM',
 'L postcentral gyrus (#41) GM',
 'R postcentral gyrus (#42) GM',
 'L superior parietal gyrus (#43) GM',
 'R superior parietal gyrus (#44) GM',
 'L supramarginal gyrus (#45) GM',
 'R supramarginal gyurs (#46) GM',
 'L angular gyrus (#47) GM',
 'R angular gyrus (#48) GM',


- Features have various names mixing underscores, capslock, spaces, coma, etc.
- Multiple features related to MRI features that are not relevant here: can be filtered with regex

Let's homogenize columns names and rows and then filter MRI features with regex to have a better overview. Let's apply columns names homogenization also to the description file so we can still match description and variables

In [162]:
column_name_cleaning(df)
str_column_cleaning(df)
df.head()

,patient id,study,group,template,diabetes duration,htn or not,htn2,age,sex,gender,height (m),mass (kg),bmi,global gm vol,global gm vol covered in template,l superior frontal gyrus (#21) gm,r superior frontal gyrus (#22) gm,l middle frontal gyrus (#23) gm,r middle frontal gyrus (#24) gm,l inferior frontal gyrus (#25) gm,r inferior frontal gyrus (#26) gm,l precentral gyrus (#27) gm,r precentral gyrus (#28) gm,l middle orbitofrontal gyrus (#29) gm,r middle orbitofrontal gyrus (#30) gm,l lateral orbitofrontal gyrus (#31) gm,r lateral orbitofrontal gyrus (#32) gm,l gyrus rectus (#33) gm,r gyrus rectus (#34) gm,l postcentral gyrus (#41) gm,r postcentral gyrus (#42) gm,l superior parietal gyrus (#43) gm,r superior parietal gyrus (#44) gm,l supramarginal gyrus (#45) gm,r supramarginal gyurs (#46) gm,l angular gyrus (#47) gm,r angular gyrus (#48) gm,l precuneus (#49) gm,r precuneus (#50) gm,l superior occipital gyrus (#61) gm,r superior occipital gyrus (#62) gm,l middle occipital gyrus (#63) gm,r middle occipital gyrus (#64) gm,l inferior occipital gyrus (#65) gm,r inferior occipital gyrus (#66) gm,l cuneus (#67) gm,r cuneus (#68) gm,l superior temporal gyrus (#81) gm,r superior temporal gyrus (#82) gm,l middle temporal gyrus (#83) gm,r middle temporal gyrus (#84) gm,l inferior temporal gyrus (#85) gm,r inferior temporal gyrus (#86) gm,l parahippocampal gyrus (#87) gm,r parahippocampal gyrus (#88) gm,l lingual gyrus (#89) gm,r lingual gyrus (#90) gm,l fusiform gyrus (#91) gm,r fusiform gyrus (#92) gm,l insular cortex (#101) gm,r insular cortex (#102) gm,l cingulate gyrus (#121) gm,r cingulate gyrus (#122) gm,l caudate (#161) gm,r caudate (#162) gm,l putamen (#163) gm,r putamen (#164) gm,l hippocampus (#165) gm,r hippocampus (#166) gm,cerebellum (#181) gm,brain stem (#182) gm,global wm vol,global wm vol covered in template,l superior frontal gyrus (#21) wm,r superior frontal gyrus (#22) wm,l middle frontal gyrus (#23) wm,r middle frontal gyrus (#24) wm,l inferior frontal gyrus (#25) wm,r inferior frontal gyrus (#26) wm,l precentral gyrus (#27) wm,r precentral gyrus (#28) wm,l middle orbitofrontal gyrus (#29) wm,r middle orbitofrontal gyrus (#30) wm,l lateral orbitofrontal gyrus (#31) wm,r lateral orbitofrontal gyrus (#32) wm,l gyrus rectus (#33) wm,r gyrus rectus (#34) wm,l postcentral gyrus (#41) wm,r postcentral gyrus (#42) wm,l superior parietal gyrus (#43) wm,r superior parietal gyrus (#44) wm,l supramarginal gyrus (#45) wm,r supramarginal gyurs (#46) wm,l angular gyrus (#47) wm,r angular gyrus (#48) wm,l precuneus (#49) wm,r precuneus (#50) wm,l superior occipital gyrus (#61) wm,r superior occipital gyrus (#62) wm,l middle occipital gyrus (#63) wm,r middle occipital gyrus (#64) wm,l inferior occipital gyrus (#65) wm,r inferior occipital gyrus (#66) wm,l cuneus (#67) wm,r cuneus (#68) wm,l superior temporal gyrus (#81) wm,r superior temporal gyrus (#82) wm,l middle temporal gyrus (#83) wm,r middle temporal gyrus (#84) wm,l inferior temporal gyrus (#85) wm,r inferior temporal gyrus (#86) wm,l parahippocampal gyrus (#87) wm,r parahippocampal gyrus (#88) wm,l lingual gyrus (#89) wm,r lingual gyrus (#90) wm,l fusiform gyrus (#91) wm,r fusiform gyrus (#92) wm,l insular cortex (#101) wm,r insular cortex (#102) wm,l cingulate gyrus (#121) wm,r cingulate gyrus (#122) wm,l caudate (#161) wm,r caudate (#162) wm,l putamen (#163) wm,r putamen (#164) wm,l hippocampus (#165) wm,r hippocampus (#166) wm,cerebellum (#181) wm,brain stem (#182) wm,global csf vol,global csf vol covered in template,l superior frontal gyrus (#21) csf,r superior frontal gyrus (#22) csf,l middle frontal gyrus (#23) csf,r middle frontal gyrus (#24) csf,l inferior frontal gyrus (#25) csf,r inferior frontal gyrus (#26) csf,l precentral gyrus (#27) csf,r precentral gyrus (#28) csf,l middle orbitofrontal gyrus (#29) csf,r middle orbitofrontal gyrus (#30) csf,l lateral orbitofrontal gyrus (#31) csf,r lateral orbitofrontal gyrus (#32) csf,l gyrus rectus (#33) csf,r gyrus rectus (

In [163]:
column_name_cleaning(df_desc)
str_column_cleaning(df_desc)
df_desc.head()

,subject number,s0328,s0342,variable description,units
0,NaN,NaN,NaN,NaN,NaN
1,study,ge75,ge75,study number,NaN
2,completed visits status,completed,completed,completed study status,NaN
3,htn patient medical history,no,no,hypertension patient history,NaN
4,group 2,control,dm,group control diabetes,NaN


In [171]:
brain_measures_re = re.compile('(^r )|(^l )|(#[0-9]+)|(perfusion)')
brain_measures_columns = list(filter(brain_measures_re.search, df.columns))
len(brain_measures_columns)
brain_measures_columns

['l superior frontal gyrus (#21) gm',
 'r superior frontal gyrus (#22) gm',
 'l middle frontal gyrus (#23) gm',
 'r middle frontal gyrus (#24) gm',
 'l inferior frontal gyrus (#25) gm',
 'r inferior frontal gyrus (#26) gm',
 'l precentral gyrus (#27) gm',
 'r precentral gyrus (#28) gm',
 'l middle orbitofrontal gyrus (#29) gm',
 'r middle orbitofrontal gyrus (#30) gm',
 'l lateral orbitofrontal gyrus (#31) gm',
 'r lateral orbitofrontal gyrus (#32) gm',
 'l gyrus rectus (#33) gm',
 'r gyrus rectus (#34) gm',
 'l postcentral gyrus (#41) gm',
 'r postcentral gyrus (#42) gm',
 'l superior parietal gyrus (#43) gm',
 'r superior parietal gyrus (#44) gm',
 'l supramarginal gyrus (#45) gm',
 'r supramarginal gyurs (#46) gm',
 'l angular gyrus (#47) gm',
 'r angular gyrus (#48) gm',
 'l precuneus (#49) gm',
 'r precuneus (#50) gm',
 'l superior occipital gyrus (#61) gm',
 'r superior occipital gyrus (#62) gm',
 'l middle occipital gyrus (#63) gm',
 'r middle occipital gyrus (#64) gm',
 'l infe

In [169]:
print(len(brain_measures_columns))
print(df.shape[1] - len(brain_measures_columns))

915
196


In [170]:
print(len(brain_measures_columns))
print(df.shape[1] - len(brain_measures_columns))

915
196


By filtering those we have now 196 features left

In [172]:
df.drop(brain_measures_columns ,axis='columns', inplace=True)
df.shape #check

(88, 196)

In [174]:
list(df.columns)

['patient id',
 'study',
 'group',
 'template',
 'diabetes duration',
 'htn or not',
 'htn2',
 'age',
 'sex',
 'gender',
 'height (m)',
 'mass (kg)',
 'bmi',
 'global gm vol',
 'global gm vol covered in template',
 'global wm vol',
 'global wm vol covered in template',
 'global csf vol',
 'global csf vol covered in template',
 'global gm covered in template/icv',
 'global wm covered in template/icv',
 'global csf covered in template/icv',
 'global intracranial vol (unit ml)',
 'global gm / icv',
 'global wm / icv',
 'global csf / icv',
 'wmh unregistered',
 'global intracranial vol (cm^3)',
 'wmh registered',
 'wmh registered masked',
 'global vasoreactivity',
 'ethnicity',
 'race',
 'race2',
 'dm nondm stroke',
 'previous tobacco use',
 'current tobacco use',
 'packyears',
 'years',
 'previous alcohol use',
 'current alcohol use',
 'alcohol dose/week',
 'neuropathy autonomic symptoms',
 'dizziness autonomic symptoms',
 'numbness autonomic symptoms',
 'painful feet autonomic symptoms',

In [ ]:
#scores_test = df.loc[:, df.columns.str.contains('(DS)|(^HVLT)|(^VF)|(^Clock)|(^TM)|(Score)')].columns
#pd.unique(df_desc["Variable description"].where(df_desc["SUBJECT NUMBER"].isin(scores_test)))

In [ ]:
filter_1 = '(^R)|(^L)|(Perfusion)|(#[0-9]+)|(^global)|(DS)|(^HVLT)|(^VF)|(^Clock)|(^TM)|(Score)'
df_v1 = df.loc[:, ~df.columns.str.contains(filter_1)]
list(df_v1.columns)

In [ ]:
df_v1.shape

Let's try to filter on variables with many missing values

In [ ]:
missing_values_counts_v1 = df_v1.isna().sum().sort_values(ascending=False)
missing_values_counts_v1.shape #checking

In [ ]:
fig = px.bar(missing_values_counts_v1, height=800, width=2000, labels={'index': 'features'})
fig.update_layout(font_size=10, bargroupgap=0.3)
fig.show()

In [ ]:
len(df_v1)*1/3

In [ ]:
df_v1.shape

In [ ]:
(missing_values_counts_v1 > len(df_v1)*1/3).sum() 
#34 features have a more than 29 missing values

In [ ]:
df_v2 = df_v1.dropna(thresh=(len(df_v1)*2/3), axis='columns')

In [ ]:
missing_values_counts_v2 = df_v2.isna().sum().sort_values(ascending=False)
missing_values_counts_v2.shape #checking

In [ ]:
fig = px.bar(missing_values_counts_v2, height=800, width=2000, labels={'index95': 'features'})
fig.update_layout(font_size=10, bargroupgap=0.3)
fig.show()

In [ ]:
df_desc[df_desc['SUBJECT NUMBER'].isin(df_v2.columns.to_list())] #missing variable description

"MRI YES OR NO" & "Eye Scan (Y/N)" can be excluded as well

In [ ]:
df_v3 = df_v2.drop(axis='columns', labels=["MRI YES OR NO", "Eye Scan (Y/N)"])

In [ ]:
print(df_v3.shape)
print(df_v2.shape)

Because we want to merge some datasets together, only unrelevant features are excluded here. Further cleaning and exploration is performed on merge.ipynb

In [ ]:
df_v3.to_csv("../data/cerebral_perfusion_data/exploration/cerebral_perfusion_diabetes_v1.csv", index=False)